In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 读取数据集：no-filter数据、ground-truth数据集、运动控制U、测量集合Z
X_no_filter = np.load('./dataset/no_filter.npy')
X_ground_truth = np.load('./dataset/ground_truth.npy')
U = np.load('./dataset/u.npy')
Z = np.load('./dataset/z.npy')

# 无迹卡尔曼滤波

## 1. 无迹变换

无迹卡尔曼滤波通过无迹变换（Unscented Transform）来线性化。它不是通过泰勒级数展开去近似函数$g$，而是由UKF明确地从高斯中提取所谓的$\sigma$点，并将他们经过函数$g$进行变换。

通常情况下，这些$\sigma$点位于均值处及对称分布于主轴的协方差处（每维两个）。对于具有均值$\mu$和$\Sigma$的$n$维高斯分布，结果$2n+1$个$\sigma$点$\chi^{[i]}$根据如下规律进行选择：
$$\begin{align}
\chi^{[0]} &= \mu \\
\chi^{[i]} &= \mu + (\sqrt{(n + \lambda ) \Sigma})_i & i=1, \cdots , n \\
\chi^{[i]} &= \mu - (\sqrt{(n + \lambda ) \Sigma})_{i-n} & i=n+1, \cdots , 2n
\end{align}$$

式中，$\lambda = \alpha^2 (n + \kappa) - n$，$\alpha$和$\kappa$为确定$\sigma$点分布在均值多远范围内的比例参数。

每个$\sigma$点$\chi^{[i]}$有两个与之相关的权值。一个权值$\omega_m^{[i]}$，计算均值的时候使用。另一个权值$\omega_c^{[i]}$，计算高斯的协方差时使用。
$$\begin{align}
\omega_m^{[0]} &= \frac {\lambda}{n+ \lambda } \\
\omega_c^{[0]} &= \frac {\lambda}{n+ \lambda } + ( 1 - \alpha^2 + \beta ) \\
\omega_m^{[i]} &= \omega_c^{[i]} = \frac {1}{2(n+ \lambda )} \quad where \quad i = 1, \cdots , 2n \\
\end{align}$$

选择参数$\beta$对高斯表示的附加的（较高阶）分布信息进行编码。如果分布是精确的高斯分布，则$\beta = 2$是最佳选择。

$\sigma$点经过函数$g$变换，来探测$g$如何改变了高斯分布的形状。
$$y^{[i]} = g(\chi^{[i]})$$

结果高斯分布的参数$(\mu' , \Sigma')$由映射的$\sigma$点$y^{[i]}$获得：
$$\begin{align}
\mu' &= \sum_{i=0}^{2n} \omega_m^{[i]} y^{[i]} \\
\Sigma' &= \sum_{i=0}^{2n} \omega_c^{[i]} (y^{[i]} - \mu')(y^{[i]} - \mu')^T
\end{align}$$

## 2. 无迹卡尔曼滤波算法

----
01: **Algorithm Unscented_Kalman_filter**$( \mu_{t-1}, \Sigma_{t-1}, u_t, z_t )$**:**  

02: &emsp;&emsp;$\chi_{t-1} = ( \mu_{t-1} \quad \mu_{t-1} + \gamma \sqrt{\Sigma_{t-1}} \quad \mu_{t-1} - \gamma \sqrt{\Sigma_{t-1}})$  

03: &emsp;&emsp;$\bar{\chi}_t^* = g(\mu_t, \chi_{t-1})$  

04: &emsp;&emsp;$\bar{\mu}_t = \sum_{i=0}^{2n} \omega_m^{[i]} \bar{\chi}_t^{*[i]}$  

05: &emsp;&emsp;$\bar{\Sigma}_t = \sum_{i=0}^{2n} \omega_c^{[i]} (\bar{\chi}_t^{*[i]} - \bar{\mu}_t)(\bar{\chi}_t^{*[i]} - \bar{\mu}_t)^T + R_t$  

06: &emsp;&emsp;$\bar{\chi}_t = ( \bar{\mu}_t \quad \bar{\mu}_t + \gamma \sqrt{\bar{\Sigma}_t} \quad \bar{\mu}_t - \gamma \sqrt{\bar{\Sigma}_t})$  

07: &emsp;&emsp;$\bar Z_t = h(\bar{\chi}_t)$  

08: &emsp;&emsp;$\widehat{z}_t = \sum_{i=0}^{2n} \omega_m^{[i]} \bar Z_t^{[i]}$  

09: &emsp;&emsp;$S_t = \sum_{i=0}^{2n} \omega_c^{[i]} (\bar Z_t^{[i]} - \widehat z_t) (\bar Z_t^{[i]} - \widehat z_t)^T + Q_t$  

10: &emsp;&emsp;$\bar{\Sigma}_t^{x,z} = \sum_{i=0}^{2n} \omega_c^{[i]} (\bar {\chi}_t^{[i]} - \bar {\mu}_t)(\bar {Z}_t^{[i]} - \widehat {z}_t)^T$  

11: &emsp;&emsp;$K_t = \bar {\Sigma}_t^{x,z} S_t^{-1}$  

12: &emsp;&emsp;$\mu_t = \bar {\mu}_t + K_t (z_t - \widehat z_t)$  

13: &emsp;&emsp;$\Sigma_t = \bar {\Sigma}_t - K_t S_t K_t^T$  

14: &emsp;&emsp;**return**$\quad \mu_t , \Sigma_t$  

----

In [3]:
# 设定UKF的超参数

# 状态Xt的维度
n_ukf = 3

# alpha和kappa为确定sigma点分布在均值多远范围内的的比例参数
alpha_ukf = 1.0
kappa_ukf = 0.8

# 计算有alpha和kappa确定的超参数lambda_ukf
lambda_ukf = alpha_ukf ** 2 * (n_ukf + kappa_ukf) - n_ukf

# 根据lambda的值计算gamma的值
gamma_ukf = np.sqrt(n_ukf + lambda_ukf)

# 超参数beta参与生成协方差的系数，对高斯表示的附加的（高阶）信息进行编码。
# 如果分布是精确地高斯分布，则beta=2是较好的选择
beta_ukf = 2.0

lambda_ukf

0.7999999999999998

In [4]:
# 生成均值与协方差系数
# 涉及UKF算法的第4/5/8/9/10行
def weight_sigmaPoint_ukf():
    weight = 0.5 / (n_ukf + lambda_ukf)
    weight_mean = np.ones(1 + n_ukf * 2) * weight
    weight_cov  = np.ones(1 + n_ukf * 2) * weight
    weight_mean[0] = lambda_ukf / (lambda_ukf + n_ukf)
    weight_cov[0]  = lambda_ukf / (lambda_ukf + n_ukf) + (1 - alpha_ukf ** 2 + beta_ukf)
    return weight_mean.reshape((-1,1)), weight_cov.reshape((-1,1))

# test code
weight_sigmaPoint_ukf()

(array([[ 0.21052632],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895]]), array([[ 2.21052632],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895],
        [ 0.13157895]]))

In [5]:
# 通过均值mu和协方差矩阵Sigma产生sigma点集，点集以(n_ukf * 2 + 1) by 3的矩阵形式返回
# 涉及UKF算法的第2/6行
def generate_sigma_points(mu, sigma):
    # 将mu的形状重新调整为向量
    mu = np.squeeze(mu).reshape((1,3))
    
    # 计算sigma矩阵的平方根
    e, v = np.linalg.eigh(sigma)
    sqrt_sigma = np.dot(v, np.diag(np.sqrt(e)))
    
    # 为了便于计算，根据返回矩阵的格式，将sigma矩阵平方根进行转置
    sqrt_sigma = sqrt_sigma.T
    
    # 生成正方向的sigma点
    positive_sigma_points = mu + gamma_ukf * sqrt_sigma
    negative_sigma_points = mu - gamma_ukf * sqrt_sigma
    
    sigma_points = np.vstack([mu , positive_sigma_points, negative_sigma_points])
    
    return sigma_points

# test code
m = np.array([2, 1, .5, 1,3,0.7,.5,0.7,5]).reshape((3, 3))
generate_sigma_points(np.array([0, 0, np.pi/4]), m)

array([[ 0.        ,  0.        ,  0.78539816],
       [ 1.95636433, -1.19110151,  0.74549493],
       [ 1.61535726,  2.70366259, -0.72097732],
       [ 1.0785451 ,  1.63446805,  4.8755376 ],
       [-1.95636433,  1.19110151,  0.8253014 ],
       [-1.61535726, -2.70366259,  2.29177365],
       [-1.0785451 , -1.63446805, -3.30474128]])

运动模型：

假设机器人的运动控制是向前移动$u_t$个单位，因此移动后机器人的期望位置将为：
$$\begin{bmatrix}
x' \\ y' \\ \theta'
\end{bmatrix} = \begin{bmatrix}
x + u_t cos \theta \\ y + u_t sin \theta \\ \theta
\end{bmatrix}$$

In [6]:
# 运动迁移方程的向量化版本
# 涉及UKF算法的第3行
def motion_model(sigma_points, u):
    xs = sigma_points[:,0].reshape((-1,1))
    ys = sigma_points[:,1].reshape((-1,1))
    thetas = sigma_points[:,2].reshape((-1,1))
    
    xs_motion = xs + u * np.cos(thetas)
    ys_motion = ys + u * np.sin(thetas)
    thetas_motion = thetas
    
    sigma_points_motion = np.hstack([xs_motion, ys_motion, thetas_motion])
    return sigma_points_motion

# test code
motion_model(generate_sigma_points(np.array([0, 0, np.pi/4]), m),2)

array([[ 1.41421356,  1.41421356,  0.78539816],
       [ 3.42586886,  0.16556958,  0.74549493],
       [ 3.11767914,  1.38342436, -0.72097732],
       [ 1.40339673, -0.33897347,  4.8755376 ],
       [-0.59969324,  2.66060604,  0.8253014 ],
       [-2.93559549, -1.20134071,  2.29177365],
       [-3.05198661, -1.30961642, -3.30474128]])

测量模型：
$$h(\begin{bmatrix} x \\ y \\ \theta \end{bmatrix}) = 
\begin{bmatrix} z_{distance} \\ z_{\theta} \end{bmatrix} = 
\begin{bmatrix} \sqrt{x^2 + y^2} \\ \theta \end{bmatrix} $$

In [7]:
# 测量模型的向量化版本
# 涉及UKF算法的第7行
def measurement_model(sigma_points):
    xs = sigma_points[:,0].reshape((-1,1))
    ys = sigma_points[:,1].reshape((-1,1))
    thetas = sigma_points[:,2].reshape((-1,1))
    
    z_distance = np.sqrt(xs ** 2 + ys ** 2)
    z_theta = thetas
    
    sigma_points_measurement = np.hstack([z_distance, z_theta])
    return sigma_points_measurement
    
measurement_model(generate_sigma_points(np.array([0, 0, np.pi/4]), m))

array([[ 0.        ,  0.78539816],
       [ 2.29043319,  0.74549493],
       [ 3.14947146, -0.72097732],
       [ 1.95825058,  4.8755376 ],
       [ 2.29043319,  0.8253014 ],
       [ 3.14947146,  2.29177365],
       [ 1.95825058, -3.30474128]])

In [32]:
# 定义UKF算法
def unscented_kalman_filter(mu, Sigma, u, z):
    # 计算sigma点的系数
    weight_m, weight_c = weight_sigmaPoint_ukf()
    
    # line 2
    sigma_points_original = generate_sigma_points(mu, Sigma)
    
    # line 3
    sigma_points_motion_model = motion_model(sigma_points_original, u)
    
    # line 4
    mu_after_motion = np.sum(weight_m * sigma_points_motion_model, axis=0, keepdims=True).T
    
    # line 5
    R = np.diag([0.09, 0.09, 0.09])
    temp = sigma_points_motion_model - mu_after_motion.T
    Sigma_after_motion = np.dot((weight_c * temp).T, temp) + R
    
    # line 6
    sigma_points_before_measure = generate_sigma_points(mu_after_motion,
                                                      Sigma_after_motion)
    
    # line 7
    z_sigma_points_after_measure = measurement_model(sigma_points_before_measure)
    
    # line 8
    z_hat = np.sum(weight_m * z_sigma_points_after_measure, axis=0, keepdims=True).T
    
    # line 9
    Q = np.diag([0.04, 0.04])
    temp = z_sigma_points_after_measure - z_hat.T
    S = np.dot((weight_c * temp).T, temp) + Q
    
    # line 10
    temp1 = sigma_points_before_measure - mu_after_motion.T
    temp2 = z_sigma_points_after_measure - z_hat.T
    Sigma_x_z = np.dot((weight_c * temp1).T, temp2)
    
    # line 11
    K = np.dot(Sigma_x_z, np.linalg.inv(S))
    
    # line 12
    mu_after_measure = mu_after_motion + np.dot(K, z - z_hat)
    
    # line 13
    Sigma_after_measure = Sigma_after_motion + np.dot(np.dot(K, S), K.T)
    
    return mu_after_measure, Sigma_after_measure

unscented_kalman_filter(np.array([0,0,np.pi/4]).reshape((-1,1)), np.diag([0.01, 0.01, 0.01]),
                        U[0], Z[0])

(array([[ 0.24225468],
        [ 0.23786789],
        [ 0.72846889]]), array([[ 0.1289187 ,  0.02573776, -0.00660487],
        [ 0.02573776,  0.1289187 ,  0.00660487],
        [-0.00660487,  0.00660487,  0.17142857]]))

In [22]:
Z[0]

array([[ 0.31201862],
       [ 0.70569718]])